In [ ]:
## INSTALL PACKAGE
!pip install -q markdown openai gdown xformers ctransformers tokenizers transformers accelerate langchain chainlit sentence_transformers chromadb unstructured PyPDF2 pypdf bitsandbytes faiss_cpu faiss_gpu huggingface_hub hf_transfer optimum -q
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  -q # Use cu117 if on CUDA 11.7

In [ ]:
### LOAD LIBRARY
from torch import cuda, bfloat16, float16
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig, pipeline, TextStreamer

In [ ]:
%%bash
# Download model
mkdir -p Llama-7B-Chat-GPTQ
HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/Llama-2-7B-Chat-GPTQ --local-dir Llama-7B-Chat-GPTQ --local-dir-use-symlinks False

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MODEL_ID = "./Llama-7B-Chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={"temperature": 0})

In [ ]:
llm("Explain the difference between ChatGPT and open source LLMs in a couple of lines.")

"\nChatGPT is a proprietary language model developed by Meta AI, while open source LLMs are models that are available for anyone to use and modify under an open-source license. This means that the code underlying these models is freely available for anyone to access, modify, and distribute, whereas ChatGPT's code is not publicly available."

In [ ]:
### Prompt Template
prompt_template = '''
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{question} [/INST]

'''

prompt = PromptTemplate(
    input_variables=["question"],
    template=prompt_template,
)

In [ ]:
question = "Explain what are Deep Neural Networks in 2-3 sentences"
print(prompt.format(question=question))


<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Explain what are Deep Neural Networks in 2-3 sentences [/INST]




In [ ]:
llm(prompt.format(question=question))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'Deep neural networks (DNNs) are a type of artificial neural network (ANN) designed to recognize complex patterns in data by mimicking the structure and function of the human brain. DNNs consist of multiple layers of interconnected nodes or "neurons," each of which processes inputs from the previous layer and passes them on to the next layer. By stacking these layers together, DNNs can learn to perform increasingly sophisticated tasks such as image recognition, speech recognition, and natural language processing.'

In [ ]:
## Create Chain01
from langchain.chains import LLMChain
chain01 = LLMChain(llm=llm, prompt=prompt)
result01 = chain01.run(question=question)
print(result01.strip())

In [ ]:
## Create Chain02
### Prompt Template for summary
prompt_template = "<s>[INST] Use the summary {summary} and give 3 examples of practical applications with 1 sentence explaining each [/INST]"

prompt02 = PromptTemplate(
    input_variables=["summary"],
    template=prompt_template,
)
chain02 = LLMChain(llm=llm, prompt=prompt02)
print(prompt02.format(summary=result01))
result02 = chain02.run(result01)
print(result02.strip())

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Deep neural networks (DNNs) are a type of artificial neural network (ANN) that are designed to model complex patterns in data by using multiple layers of interconnected nodes or "neurons." DNNs can be trained to perform a variety of tasks, such as image recognition, natural language processing, and speech recognition, by optimizing their weights and biases to minimize the error between the network's predictions and the true labels. By stacking multiple layers of neurons, DNNs can learn to represent and manipulate high-dimensional data representations, making them particularly useful for tasks like image classification and language translation.


In [ ]:
## Chaining Chains
## multi_chain
multi_chain = SimpleSequentialChain(chains=[chain01, chain02], verbose=True)
result_mutiple = multi_chain.run(question)
print(result_mutiple.strip())

<s>[INST] Use the summary Deep neural networks (DNNs) are a type of artificial neural network (ANN) that are designed to model complex patterns in data by using multiple layers of interconnected nodes or "neurons." DNNs can be trained to perform a variety of tasks, such as image recognition, natural language processing, and speech recognition, by optimizing their weights and biases to minimize the error between the network's predictions and the true labels. By stacking multiple layers of neurons, DNNs can learn to represent and manipulate high-dimensional data representations, making them particularly useful for tasks like image classification and language translation. and give 3 examples of practical applications with 1 sentence explaining each [/INST]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Sure! Here is a summary of deep neural networks (DNNs), along with three examples of practical applications:
Summary: Deep neural networks (DNNs) are a type of artificial neural network (ANN) designed to model complex patterns in data by using multiple layers of interconnected nodes or "neurons." They can be trained to perform various tasks like image recognition, natural language processing, and speech recognition by optimizing their weights and biases to minimize the error between the network's predictions and the true labels.
Practical Applications:
1. Image Recognition: DNNs can be trained to recognize images of objects, people, and scenes by learning to identify patterns in the visual data. For example, a DNN might be trained to classify images of dogs, cats, and cars based on their features such as color, shape, and texture.
2. Natural Language Processing (NLP): DNNs can be used to process and analyze natural language data, such as text or speech, allowing for tasks like language



> Entering new SimpleSequentialChain chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Deep neural networks (DNNs) are a type of artificial neural network (ANN) designed to solve complex problems by mimicking the structure and function of the human brain. DNNs consist of multiple layers of interconnected nodes (neurons) that process and transmit information, allowing them to learn and represent intricate patterns in data. By stacking these layers, DNNs can learn hierarchical representations of input data, enabling them to perform tasks such as image recognition, speech recognition, and natural language processing.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
